In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP
from micrograd.draw_dot import draw_dot

In [12]:
#initializing random seed
np.random.seed(1)
random.seed(1)

# makign a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples = 100, noise = 0.1)

y = y*2 - 1 #makes y -1 or 1
## WHY KARPATHY 

# visualize
#plt.scatter(X[:,0],X[:,1],c = y, s = 20, cmap = "jet")

In [4]:
# #initialize network and inputs and ygt
# test_nn = MLP(3,[10,10,1])

# #draw_dot(nn(x))

# xs = [
#     [2.0,3.0,-1.0],
#     [3.0,-1.0,0.5],
#     [0.5,1.0,1.0],
#     [1.0,1.0,-1.0]
# ]

# ys = [1.0,0.0,0.0,1.0] #desired targets

# epochs = 80
# for k in range(epochs):
#     #forward pass
#     ypred = [ test_nn(x) for x in xs]
#     loss = sum(((yout - ygt)**2 for yout, ygt in zip(ypred,ys)))

#     # I WAS RIGHT - NEED TO ZERO_GRAD() before backward pass
#     for p in test_nn.parameters():
#         p.grad = 0.0

#     #backward pass
#     loss.backward()

#     # update
#     for p in test_nn.parameters():
#         step_size = 0.001
#         p.data += -step_size * p.grad

#     #print results
#     print(k,loss.data)

# ypred

In [13]:
from test_micrograd.engine import Value
from test_micrograd.nn import Neuron, Layer, MLP
# #from test_micrograd.draw_dot import draw_dot
# from test_micrograd.test_engine import test_sanity_check, test_more_ops

# test_sanity_check()
# test_more_ops()

# #KARPATHIES CODE DOESNT WORK EITHER!!!!

In [18]:
model = MLP(2, [16, 16, 1])
print(model)

MLP of [Layer of [ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2), ReLUNeuron(2)], Layer of [ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16), ReLUNeuron(16)], Layer of [LinearNeuron(16)]]


In [15]:
# loss function
def loss(batch_size=None):
    
    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(total_loss, acc)

Value(data=0.6195071936644958, grad=0) 0.76


In [16]:
# optimization
for k in range(100):
    
    # forward
    total_loss, acc = loss()
    
    # backward
    model.zero_grad()
    total_loss.backward()
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 0.6195071936644958, accuracy 76.0%
step 1 loss 0.5528191443450593, accuracy 79.0%
step 2 loss 0.35368878171951396, accuracy 84.0%
step 3 loss 0.30245712775594286, accuracy 87.0%
step 4 loss 0.2483273764260679, accuracy 87.0%
step 5 loss 0.19587362870477665, accuracy 88.0%
step 6 loss 0.26460586015134674, accuracy 96.0%
step 7 loss 0.370531140276335, accuracy 87.0%
step 8 loss 0.28969923504127076, accuracy 87.0%
step 9 loss 0.16666059179383513, accuracy 92.0%
step 10 loss 0.16318107610605048, accuracy 94.0%
step 11 loss 0.19418010587856308, accuracy 92.0%
step 12 loss 0.20666640373284637, accuracy 91.0%
step 13 loss 0.13598454546532374, accuracy 95.0%
step 14 loss 0.0774522769581141, accuracy 97.0%
step 15 loss 0.07851779746397006, accuracy 99.0%
step 16 loss 0.08916529295158604, accuracy 98.0%
step 17 loss 0.16088252677768913, accuracy 94.0%
step 18 loss 0.07569935361357283, accuracy 97.0%
step 19 loss 0.05918533602569771, accuracy 98.0%
step 20 loss 0.05336125552745471, ac

KeyboardInterrupt: 

In [6]:
# define loss function

def loss(batch_size = None):

    #inline DataLoader
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb,yb = X[ri], y[ri]

    inputs = [list(map(Value,xrow)) for xrow in Xb] #turns all of the inputs into Value objects

    #forward the mode to get scores

    scores = list(map(model, inputs)) #more python way of iterating over each element in the input list

    # svm "max-margin" LOSS
    losses = [(1 + -yi*scorei).relu() for yi,scorei in zip(yb,scores)]
    data_loss = sum(losses) * (1.0 / len(losses)) #takes average of loss

    #L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters())) #L**2 term
    total_loss = data_loss + reg_loss 

    #can also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi,scorei in zip(yb,scores)]
    
    #return both
    return total_loss, sum(accuracy)/len(accuracy)

total_loss,acc = loss()

print(total_loss,acc)

#draw_dot(total_loss)


Value(data=0.7362259379254892, grad=0) 0.76


In [8]:
# optimization

for k in range(100):
    #forward
    total_loss, acc = loss()

    #backward
    for p in model.parameters():
        p.grad = 0
    total_loss.backward()

    #update (sgd) with learning
    learning_rate = 1.0 - (0.9*k/100)
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 0.7362259379254892, accuracy 76.0%
step 1 loss 0.5120356468409522, accuracy 50.0%
step 2 loss 0.511029180423619, accuracy 50.0%
step 3 loss 0.5110248485869767, accuracy 50.0%
step 4 loss 0.5110205581334084, accuracy 50.0%
step 5 loss 0.5110163090158466, accuracy 50.0%
step 6 loss 0.5110121011876885, accuracy 50.0%
step 7 loss 0.5110079346027954, accuracy 50.0%
step 8 loss 0.5110038092154912, accuracy 50.0%
step 9 loss 0.5109997249805626, accuracy 50.0%
step 10 loss 0.5109956818532573, accuracy 50.0%
step 11 loss 0.5109916797892836, accuracy 50.0%
step 12 loss 0.5109877187448099, accuracy 50.0%
step 13 loss 0.510983798676463, accuracy 50.0%
step 14 loss 0.5109832768350713, accuracy 50.0%
step 15 loss 0.5109797768761287, accuracy 50.0%
step 16 loss 0.5109759782019433, accuracy 50.0%
step 17 loss 0.5109722203487067, accuracy 50.0%
step 18 loss 0.5109685032753154, accuracy 50.0%
step 19 loss 0.5109648269411201, accuracy 50.0%
step 20 loss 0.5109611913059255, accuracy 50.0%
step

KeyboardInterrupt: 

In [ ]:
#visualize the decision boundary

h = 0.25
x_min,x_max = 